In [2]:
!pip install pygame

In [ ]:
import pygame
import sys
import random

pygame.init()

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Simple Games")

clock = pygame.time.Clock()

def pong():
    paddle_width, paddle_height = 10, 100
    ball_size = 20
    paddle_speed = 10
    ball_speed_x, ball_speed_y = 5, 5

    player = pygame.Rect(SCREEN_WIDTH - 20, SCREEN_HEIGHT // 2 - paddle_height // 2, paddle_width, paddle_height)
    opponent = pygame.Rect(10, SCREEN_HEIGHT // 2 - paddle_height // 2, paddle_width, paddle_height)
    ball = pygame.Rect(SCREEN_WIDTH // 2 - ball_size // 2, SCREEN_HEIGHT // 2 - ball_size // 2, ball_size, ball_size)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return

        keys = pygame.key.get_pressed()
        if keys[pygame.K_UP] and player.top > 0:
            player.y -= paddle_speed
        if keys[pygame.K_DOWN] and player.bottom < SCREEN_HEIGHT:
            player.y += paddle_speed

        if ball.top <= 0 or ball.bottom >= SCREEN_HEIGHT:
            ball_speed_y *= -1
        if ball.left <= 0 or ball.right >= SCREEN_WIDTH:
            ball_speed_x *= -1
        if ball.colliderect(player) or ball.colliderect(opponent):
            ball_speed_x *= -1

        opponent.y += ball_speed_y if opponent.centery < ball.centery else -ball_speed_y

        ball.x += ball_speed_x
        ball.y += ball_speed_y

        screen.fill(BLACK)
        pygame.draw.rect(screen, WHITE, player)
        pygame.draw.rect(screen, WHITE, opponent)
        pygame.draw.ellipse(screen, WHITE, ball)
        pygame.draw.aaline(screen, WHITE, (SCREEN_WIDTH // 2, 0), (SCREEN_WIDTH // 2, SCREEN_HEIGHT))

        pygame.display.flip()
        clock.tick(60)

def snake():
    block_size = 20
    snake_speed = 15
    x, y = SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2
    snake = [(x, y)]
    dx, dy = 0, 0
    food = (random.randint(0, SCREEN_WIDTH - block_size) // block_size * block_size,
            random.randint(0, SCREEN_HEIGHT - block_size) // block_size * block_size)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP and dy == 0:
                    dx, dy = 0, -block_size
                elif event.key == pygame.K_DOWN and dy == 0:
                    dx, dy = 0, block_size
                elif event.key == pygame.K_LEFT and dx == 0:
                    dx, dy = -block_size, 0
                elif event.key == pygame.K_RIGHT and dx == 0:
                    dx, dy = block_size, 0

        if not (0 <= snake[0][0] < SCREEN_WIDTH and 0 <= snake[0][1] < SCREEN_HEIGHT) or len(snake) != len(set(snake)):
            return

        if snake[0] == food:
            food = (random.randint(0, SCREEN_WIDTH - block_size) // block_size * block_size,
                    random.randint(0, SCREEN_HEIGHT - block_size) // block_size * block_size)
        else:
            snake.pop()

        x += dx
        y += dy
        snake.insert(0, (x, y))

        screen.fill(BLACK)
        for segment in snake:
            pygame.draw.rect(screen, GREEN, (*segment, block_size, block_size))
        pygame.draw.rect(screen, RED, (*food, block_size, block_size))

        pygame.display.flip()
        clock.tick(snake_speed)

def maze():
    rows, cols = 20, 20
    cell_size = 30
    grid = [[0 for _ in range(cols)] for _ in range(rows)]

    def carve_passages(cx, cy):
        directions = [(0, -1), (1, 0), (0, 1), (-1, 0)]
        random.shuffle(directions)

        for dx, dy in directions:
            nx, ny = cx + dx, cy + dy

            if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] == 0:
                grid[cx][cy] = 1
                grid[nx][ny] = 1
                carve_passages(nx, ny)

    carve_passages(0, 0)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return

        screen.fill(WHITE)
        for x in range(rows):
            for y in range(cols):
                if grid[x][y] == 0:
                    pygame.draw.rect(screen, BLACK, pygame.Rect(y * cell_size, x * cell_size, cell_size, cell_size))

        pygame.display.flip()
        clock.tick(30)

def breakout():
    paddle_width, paddle_height = 100, 10
    ball_size = 20
    paddle_speed = 10
    ball_speed_x, ball_speed_y = 5, -5

    paddle = pygame.Rect(SCREEN_WIDTH // 2 - paddle_width // 2, SCREEN_HEIGHT - paddle_height - 10, paddle_width, paddle_height)
    ball = pygame.Rect(SCREEN_WIDTH // 2 - ball_size // 2, SCREEN_HEIGHT // 2 - ball_size // 2, ball_size, ball_size)

    bricks = [pygame.Rect(10 + 60 * col, 10 + 30 * row, 50, 20) for row in range(5) for col in range(13)]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and paddle.left > 0:
            paddle.x -= paddle_speed
        if keys[pygame.K_RIGHT] and paddle.right < SCREEN_WIDTH:
            paddle.x += paddle_speed

        ball.x += ball_speed_x
        ball.y += ball_speed_y

        if ball.left <= 0 or ball.right >= SCREEN_WIDTH:
            ball_speed_x *= -1
        if ball.top <= 0 or ball.colliderect(paddle):
            ball_speed_y *= -1
        if ball.bottom >= SCREEN_HEIGHT:
            return

        for brick in bricks[:]:
            if ball.colliderect(brick):
                bricks.remove(brick)
                ball_speed_y *= -1

        screen.fill(BLACK)
        pygame.draw.rect(screen, BLUE, paddle)
        pygame.draw.ellipse(screen, RED, ball)
        for brick in bricks:
            pygame.draw.rect(screen, WHITE, brick)

        pygame.display.flip()
        clock.tick(60)

def tic_tac_toe():
    grid = [None] * 9
    player = 'X'

    def check_win():
        win_combinations = [(0, 1, 2), (3, 4, 5), (6, 7, 8),
                            (0, 3, 6), (1, 4, 7), (2, 5, 8),
                            (0, 4, 8), (2, 4, 6)]
        for combo in win_combinations:
            if grid[combo[0]] == grid[combo[1]] == grid[combo[2]] and grid[combo[0]] is not None:
                return grid[combo[0]]
        return None

    def draw_grid():
        screen.fill(WHITE)
        for i in range(1, 3):
            pygame.draw.line(screen, BLACK, (0, SCREEN_HEIGHT // 3 * i), (SCREEN_WIDTH, SCREEN_HEIGHT // 3 * i), 2)
            pygame.draw.line(screen, BLACK, (SCREEN_WIDTH // 3 * i, 0), (SCREEN_WIDTH // 3 * i, SCREEN_HEIGHT), 2)

        for i, value in enumerate(grid):
            if value is not None:
                x, y = i % 3 * SCREEN_WIDTH // 3 + SCREEN_WIDTH // 6, i // 3 * SCREEN_HEIGHT // 3 + SCREEN_HEIGHT // 6
                screen.blit(pygame.font.Font(None, 144).render(value, True, BLACK), (x - 72, y - 72))

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return
            elif event.type == pygame.MOUSEBUTTONDOWN and None in grid:
                x, y = event.pos
                index = (y // (SCREEN_HEIGHT // 3)) * 3 + (x // (SCREEN_WIDTH // 3))
                if grid[index] is None:
                    grid[index] = player
                    player = 'O' if player == 'X' else 'X'
                    winner = check_win()
                    if winner:
                        print(f"{winner} wins!")
                        return
                    elif None not in grid:
                        print("It's a tie!")
                        return

        draw_grid()
        pygame.display.flip()
        clock.tick(30)

def flappy_bird():
    bird = pygame.Rect(50, SCREEN_HEIGHT // 2, 30, 30)
    bird_speed = 0
    gravity = 1
    pipe_width, pipe_gap = 100, 200
    pipes = [pygame.Rect(SCREEN_WIDTH, random.randint(100, 400), pipe_width, 400)]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    bird_speed = -10

        bird_speed += gravity
        bird.y += bird_speed

        if bird.top <= 0 or bird.bottom >= SCREEN_HEIGHT:
            return

        pipes = [pipe.move(-5, 0) for pipe in pipes]
        if pipes[0].right < 0:
            pipes.append(pygame.Rect(SCREEN_WIDTH, random.randint(100, 400), pipe_width, 400))
            pipes.pop(0)

        if bird.collidelist([pipe for pipe in pipes for pipe in [pipe, pipe.move(0, -pipe_gap - SCREEN_HEIGHT)]]) != -1:
            return

        screen.fill(BLUE)
        pygame.draw.rect(screen, (255, 255, 0), bird)
        for pipe in pipes:
            pygame.draw.rect(screen, GREEN, pipe)
            pygame.draw.rect(screen, GREEN, pipe.move(0, -pipe_gap - SCREEN_HEIGHT))

        pygame.display.flip()
        clock.tick(30)

def space_shooter():
    player = pygame.Rect(SCREEN_WIDTH // 2 - 25, SCREEN_HEIGHT - 60, 50, 50)
    bullets = []
    enemies = [pygame.Rect(random.randint(0, SCREEN_WIDTH - 50), random.randint(-100, -40), 50, 50) for _ in range(10)]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    bullets.append(player.copy())

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and player.left > 0:
            player.x -= 10
        if keys[pygame.K_RIGHT] and player.right < SCREEN_WIDTH:
            player.x += 10

        bullets = [bullet.move(0, -10) for bullet in bullets]
        bullets = [bullet for bullet in bullets if bullet.bottom > 0]

        for enemy in enemies:
            enemy.y += 5
            if enemy.top > SCREEN_HEIGHT:
                enemy.x = random.randint(0, SCREEN_WIDTH - 50)
                enemy.y = random.randint(-100, -40)

        enemies = [enemy for enemy in enemies if enemy.collidelist(bullets) == -1]

        if any(enemy.colliderect(player) for enemy in enemies):
            return

        screen.fill(BLACK)
        pygame.draw.rect(screen, BLUE, player)
        for bullet in bullets:
            pygame.draw.rect(screen, WHITE, bullet)
        for enemy in enemies:
            pygame.draw.rect(screen, RED, enemy)

        pygame.display.flip()
        clock.tick(60)

def select_game():
    print("Select a game:")
    print("1. Pong")
    print("2. Snake")
    print("3. Maze")
    print("4. Breakout")
    print("5. Tic-Tac-Toe")
    print("6. Flappy Bird")
    print("7. Space Shooter")
    
    choice = input("Enter the number of the game you want to play: ")
    if choice == '1':
        pong()
    elif choice == '2':
        snake()
    elif choice == '3':
        maze()
    elif choice == '4':
        breakout()
    elif choice == '5':
        tic_tac_toe()
    elif choice == '6':
        flappy_bird()
    elif choice == '7':
        space_shooter()
    else:
        print("Invalid choice!")

select_game()


pygame 2.6.0 (SDL 2.28.4, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
Select a game:
1. Pong
2. Snake
3. Maze
4. Breakout
5. Tic-Tac-Toe
6. Flappy Bird
7. Space Shooter
